# Web Scraping
---

## We need to collect user reviews for our data from [Metactric Games](https://www.metacritic.com/game), wiki link [here](https://en.wikipedia.org/wiki/Metacritic), specifically all the user reviews for Metacritic's Top Games. We want to focus on the most recent consoles to get a better idea of what games are popular with the more recent generation of consoles. We will be focusing on PS4, Xbox One, Switch, PC, Xbox Series X, and PS5 games.  Ex. [Metactitic Best PS4 Games of All Time](https://www.metacritic.com/browse/games/score/metascore/all/ps4/filtered?sort=desc&view=detailed) 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from datetime import datetime

### To avoid the 301 Moved Permanently status we need to set a different user agent. This will give us a 200 status code which means we are good to continue web scraping.

In [2]:
session = requests.Session()
session.headers['User-Agent'] = \
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2)\
AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'

url = 'https://www.metacritic.com/browse/games/score/metascore/all/\
ps4/filtered?sort=desc&view=detailed'
response = session.get(url)
response

<Response [200]>

### Now let's begin webscraping the top 100 page for each console. We will be scraping each game title, summary, details, and user reviews. Then we will be adding it into a list of dictionaries to then later create a dataframe to work with. Let's create a function to do all of this for us for every console.

In [8]:
def scrape_top_100_games(console):
    session = requests.Session()
    session.headers['User-Agent'] = \
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2)\
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'

    url = 'https://www.metacritic.com/browse/games/score/metascore/all/' + \
    console + '/filtered?sort=desc&view=detailed'
    response = session.get(url)
    sleep(2)
    # grab text from top 100 page
    html = response.text

    # create BS instance
    soup = BeautifulSoup(html, 'lxml')

    # create main list to store dictionaries
    video_games = []

    # iterate over each table (site split up into uneven tables)
    for table in soup.find_all('table', {'class': 'clamp-list'}):
        # in each table search for the game
        for game in table.find_all('a', {'class':'title'}):

            # create a dictionary to store game info
            video_game = {}

            # add console to dict
            video_game['console'] = console

            # grab url for game
            url = 'https://www.metacritic.com' + game.get('href')

            # add game name to dict
            video_game['video_game_name'] = game.text
            
            now = datetime.now()

            current_time = now.strftime("%H:%M:%S")
            print(f'{game.text} - Time: {current_time}')

            # establish connection to the link that has the game info and create BS instance
            session2 = requests.Session()
            sleep(1)
            session2.headers['User-Agent'] = \
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2)\
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
            response2 = session2.get(url)
            html2 = response2.text
            soup2 = BeautifulSoup(html2, 'lxml')

            # check to see if string is empty, if so wait and try again waiting longer each time
            timer = 0
            if html2 == '':
                while html2 == '':
                    if timer == 60:
                        break
                    # establish connection to the link that has the game info and create BS instance
                    session2 = requests.Session()
                    sleep(30)
                    timer += 30
                    session2.headers['User-Agent'] = \
                    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2)\
                    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
                    response2 = session2.get(url)
                    html2 = response2.text
                    soup2 = BeautifulSoup(html2, 'lxml')
            if html2 == '':
                continue

            # add game summary to dict
            try:
                video_game['summary'] = soup2.find('div', 
                                        {'class': 'module product_data product_data_summary'})\
                                        .find('span', {'class':'blurb blurb_expanded'}).text
            except:
                video_game['summary'] = soup2.find('div', 
                                        {'class': 'module product_data product_data_summary'})\
                                        .find('span', {'class':'data'}).text
            else:
                pass

            # add developer to dict
            try:
                video_game['developer'] = soup2.find('a', {'class': 'button'}).text
            except:
                pass

            # add genre to dict
            try:
                video_game['genre(s)'] = ' '.join(soup2.find('li', 
                                             {'class': 'summary_detail product_genre'}).text.split())
            except:
                pass

            # add number of players to dict
            try:
                video_game['num_players'] = ' '.join(soup2.find('li', 
                                             {'class': 'summary_detail product_players'}).text.split())
            except:
                pass

            # add esrb game rating to dict
            try:
                video_game['esrb_rating'] = ' '.join(soup2.find('li', 
                                             {'class': 'summary_detail product_rating'}).text.split())
            except:
                pass

            # add metacritic critic score
            video_game['critic_score'] = soup2.find('span',{'itemprop': 'ratingValue'}).text

            # add average user score
            try:
                video_game['avg_user_score'] = soup2.find('div', 
                                        {'class': 'metascore_w user large game positive'}).text
            except:
                try:
                    video_game['avg_user_score'] = soup2.find('div', 
                                        {'class': 'metascore_w user large game mixed'}).text
                except:
                    video_game['avg_user_score'] = soup2.find('div', 
                                        {'class': 'metascore_w user large game negative'}).text
            else:
                pass

            # establish connection to the user reviews page
            url = url + '/user-reviews'
            session3 = requests.Session()
            sleep(1)
            session3.headers['User-Agent'] = \
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2)\
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
            response3 = session3.get(url)
            html3 = response3.text
            soup3 = BeautifulSoup(html3, 'lxml')
            
            # check to see if string is empty, if so wait and try again waiting longer each time
            timer = 0
            if html3 == '':
                while html3 == '':
                    if timer == 60:
                        break
                    # establish connection to the link that has the game info and create BS instance
                    session3 = requests.Session()
                    sleep(30)
                    timer += 30
                    session3.headers['User-Agent'] = \
                    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2)\
                    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
                    response3 = session3.get(url)
                    html3 = response3.text
                    soup3 = BeautifulSoup(html3, 'lxml')
            if html3 == '':
                continue

            # check to see if review page even has a review, if not continue to next game
            if soup3.find('div', {'class':'msg msg_no_reviews'}):
                continue

            # we want to check if there are more than one page, if not scrape the page we're on
            if soup3.find('li', {'class': 'page last_page'}) == None: 

                # get the first review and score
                first_review = soup3.find('li', {'class': 'review user_review first_review'})

                # check to see if there is only 1 review on 1 page
                one_review = soup3.find('li', {'class': 'review user_review first_review last_review'})
                if one_review:
                    if one_review.find('div', {'class': 'review_body'})\
                    .find('span', {'class': 'blurb blurb_expanded'}) == None:

                        # if not grab regular review_body and score
                        video_game['user_review'] = one_review.find('div', {'class': 'review_body'}).text

                        video_game['user_score'] = one_review.find('div', {'class': 'review_grade'}).text

                        # add dict to list
                        video_games.append(video_game)
                        continue
                    # if it is "expanded" grab expanded review and score
                    else:
                        video_game['user_review'] = one_review.find('div', {'class': 'review_body'})\
                        .find('span', {'class': 'blurb blurb_expanded'}).text

                        video_game['user_score'] = one_review.find('div', {'class': 'review_grade'}).text

                        # add dict to list
                        video_games.append(video_game)
                        continue

                # check to see if the review is an "expanded" review
                elif first_review.find('div', {'class': 'review_body'})\
                .find('span', {'class': 'blurb blurb_expanded'}) == None:

                    # if not grab regular review_body and score
                    video_game['user_review'] = first_review.find('div', {'class': 'review_body'}).text

                    video_game['user_score'] = first_review.find('div', {'class': 'review_grade'}).text

                # if it is "expanded" grab expanded review and score
                else:
                    video_game['user_review'] = first_review.find('div', {'class': 'review_body'})\
                    .find('span', {'class': 'blurb blurb_expanded'}).text

                    video_game['user_score'] = first_review.find('div', {'class': 'review_grade'}).text

                # add dict to list
                video_games.append(video_game)

                # iterate over all the user reviews and grab the score and review
                for element in soup3.find_all('li', {'class': 'review user_review'}):
                    video_game_2 = video_game.copy()

                    # check to see if the review is an "expanded" review
                    if element.find('div', {'class': 'review_body'})\
                    .find('span', {'class': 'blurb blurb_expanded'}) == None:

                        # if not grab regular review_body and score
                        video_game_2['user_review'] = element.find('div', {'class': 'review_body'}).text

                        video_game_2['user_score'] = element.find('div', {'class': 'review_grade'}).text

                    # if it is "expanded" grab expanded review and score
                    else:
                        video_game_2['user_review'] = element.find('div', {'class': 'review_body'})\
                        .find('span', {'class': 'blurb blurb_expanded'}).text

                        video_game_2['user_score'] = element.find('div', {'class': 'review_grade'}).text

                    # add dict to list
                    video_games.append(video_game_2)

                # create new dict to add to list
                video_game_3 = video_game.copy()

                # get the last review and score
                # check to see if the review is an "expanded" review
                last_review = soup3.find('li', {'class': 'review user_review last_review'})

                # check to see if the review is an "expanded" review
                if last_review.find('div', {'class': 'review_body'})\
                .find('span', {'class': 'blurb blurb_expanded'}) == None:

                    # if not grab regular review_body and score
                    video_game_3['user_review'] = last_review.find('div', {'class': 'review_body'}).text

                    video_game_3['user_score'] = last_review.find('div', {'class': 'review_grade'}).text

                # if it is "expanded" grab expanded review and score
                else:
                    video_game_3['user_review'] = last_review.find('div', {'class': 'review_body'})\
                    .find('span', {'class': 'blurb blurb_expanded'}).text

                    video_game_3['user_score'] = last_review.find('div', {'class': 'review_grade'}).text

                # add dict to list
                video_games.append(video_game_3)
            # if there is more than one page scrape all the pages            
            else:
                # here we are going to iterate through all of the user review pages and scrape the reviews
                for review_page in range(0, int(soup3.find('li', {'class': 'page last_page'})
                                 .find('a', {'class': 'page_num'}).text)):
                    # check to see if we are on the first page, if so scrape user reviews
                    if review_page == 0:
                        # get the first review and score
                        first_review = soup3.find('li', {'class': 'review user_review first_review'})

                        # check to see if the review is an "expanded" review
                        if first_review.find('div', {'class': 'review_body'})\
                        .find('span', {'class': 'blurb blurb_expanded'}) == None:

                            # if not grab regular review_body and score
                            video_game['user_review'] = first_review.find('div', {'class': 'review_body'}).text

                            video_game['user_score'] = first_review.find('div', {'class': 'review_grade'}).text

                        # if it is "expanded" grab expanded review and score
                        else:
                            video_game['user_review'] = first_review.find('div', {'class': 'review_body'})\
                            .find('span', {'class': 'blurb blurb_expanded'}).text

                            video_game['user_score'] = first_review.find('div', {'class': 'review_grade'}).text

                        # add dict to list
                        video_games.append(video_game)

                        # iterate over all the user reviews and grab the score and review
                        for element in soup3.find_all('li', {'class': 'review user_review'}):
                            video_game_2 = video_game.copy()

                            # check to see if the review is an "expanded" review
                            if element.find('div', {'class': 'review_body'})\
                            .find('span', {'class': 'blurb blurb_expanded'}) == None:

                                # if not grab regular review_body and score
                                video_game_2['user_review'] = element.find('div', {'class': 'review_body'}).text

                                video_game_2['user_score'] = element.find('div', {'class': 'review_grade'}).text

                            # if it is "expanded" grab expanded review and score
                            else:
                                video_game_2['user_review'] = element.find('div', {'class': 'review_body'})\
                                .find('span', {'class': 'blurb blurb_expanded'}).text

                                video_game_2['user_score'] = element.find('div', {'class': 'review_grade'}).text

                            # add dict to list
                            video_games.append(video_game_2)

                        # create new dict to add to list
                        video_game_3 = video_game.copy()

                        # get the last review and score
                        # check to see if the review is an "expanded" review
                        last_review = soup3.find('li', {'class': 'review user_review last_review'})

                        # check to see if the review is an "expanded" review
                        if last_review.find('div', {'class': 'review_body'})\
                        .find('span', {'class': 'blurb blurb_expanded'}) == None:

                            # if not grab regular review_body and score
                            video_game_3['user_review'] = last_review.find('div', {'class': 'review_body'}).text

                            video_game_3['user_score'] = last_review.find('div', {'class': 'review_grade'}).text

                        # if it is "expanded" grab expanded review and score
                        else:
                            video_game_3['user_review'] = last_review.find('div', {'class': 'review_body'})\
                            .find('span', {'class': 'blurb blurb_expanded'}).text

                            video_game_3['user_score'] = last_review.find('div', {'class': 'review_grade'}).text

                        # add dict to list
                        video_games.append(video_game_3)

                    # if we are not on the first page connect to the next page and scrape reviews
                    # we only want the first 10 pages, some games have over 100 pages of reviews
                    # if we scraped them all we'd have too much
                    elif review_page < 10:
                        params = {'page': review_page}
                        session4 = requests.Session()
                        session4.headers['User-Agent'] = \
                        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2)\
                        AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
                        response4 = session4.get(url, params=params)
                        sleep(1)
                        html4 = response4.text
                        soup4 = BeautifulSoup(html4, 'lxml')
                        
                        # check to see if string is empty, if so wait and try again waiting longer 
                        count = 0
                        if html4 == '':
                            while html4 == '':
                                count += 1
                                if count == 3:
                                    break
                                # establish connection to the link that has the game info and create BS instance
                                session4 = requests.Session()
                                sleep(5)
                                session4.headers['User-Agent'] = \
                                'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2)\
                                AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
                                response4 = session4.get(url, params=params)
                                html4 = response4.text
                                soup4 = BeautifulSoup(html4, 'lxml')
                        if html4 == '':
                            continue

                        # get the first review and score
                        first_review = soup4.find('li', {'class': 'review user_review first_review'})

                        # check to see if the review is an "expanded" review

                        if first_review.find('div', {'class': 'review_body'})\
                        .find('span', {'class': 'blurb blurb_expanded'}) == None:

                            # if not grab regular review_body and score
                            video_game['user_review'] = first_review.find('div', {'class': 'review_body'}).text

                            video_game['user_score'] = first_review.find('div', {'class': 'review_grade'}).text

                        # if it is "expanded" grab expanded review and score
                        else:
                            video_game['user_review'] = first_review.find('div', {'class': 'review_body'})\
                            .find('span', {'class': 'blurb blurb_expanded'}).text

                            video_game['user_score'] = first_review.find('div', {'class': 'review_grade'}).text

                        # add dict to list
                        video_games.append(video_game)

                        # iterate over all the user reviews and grab the score and review
                        for element in soup4.find_all('li', {'class': 'review user_review'}):
                            video_game_2 = video_game.copy()

                            # check to see if the review is an "expanded" review
                            if element.find('div', {'class': 'review_body'})\
                            .find('span', {'class': 'blurb blurb_expanded'}) == None:

                                # if not grab regular review_body and score
                                video_game_2['user_review'] = element.find('div', {'class': 'review_body'}).text

                                video_game_2['user_score'] = element.find('div', {'class': 'review_grade'}).text

                            # if it is "expanded" grab expanded review and score
                            else:
                                video_game_2['user_review'] = element.find('div', {'class': 'review_body'})\
                                .find('span', {'class': 'blurb blurb_expanded'}).text

                                video_game_2['user_score'] = element.find('div', {'class': 'review_grade'}).text

                            # add dict to list
                            video_games.append(video_game_2)

                        # create new dict to add to list
                        video_game_3 = video_game.copy()

                        # get the last review and score
                        # check to see if the review is an "expanded" review
                        last_review = soup4.find('li', {'class': 'review user_review last_review'})

                        # check to see if the review is an "expanded" review
                        if last_review.find('div', {'class': 'review_body'})\
                        .find('span', {'class': 'blurb blurb_expanded'}) == None:

                            # if not grab regular review_body and score
                            video_game_3['user_review'] = last_review.find('div', {'class': 'review_body'}).text

                            video_game_3['user_score'] = last_review.find('div', {'class': 'review_grade'}).text

                        # if it is "expanded" grab expanded review and score
                        else:
                            video_game_3['user_review'] = last_review.find('div', {'class': 'review_body'})\
                            .find('span', {'class': 'blurb blurb_expanded'}).text

                            video_game_3['user_score'] = last_review.find('div', {'class': 'review_grade'}).text

                        # add dict to list
                        video_games.append(video_game_3)
                    # if review_page > 10 break the loop
                    else:
                        break
    
    # create a df with the list of dictionaries from all top 100 games
    df = pd.DataFrame(video_games)
    
    # save the df to a .csv file
    df.to_csv(f'./data/top_100_{console}_games.csv', index=False)
    
    # return the df to see its contents
    return df

In [9]:
scrape_top_100_games('ps4')

Red Dead Redemption 2 - Time: 23:29:36
Grand Theft Auto V - Time: 23:29:54
Persona 5 Royal - Time: 23:30:29
The Last of Us Remastered - Time: 23:30:42
God of War - Time: 23:31:37
The Last of Us Part II - Time: 23:32:02
Persona 5 - Time: 23:32:34
Metal Gear Solid V: The Phantom Pain - Time: 23:32:50
Uncharted 4: A Thief's End - Time: 23:33:05
Journey - Time: 23:33:19
Bloodborne - Time: 23:33:26
Undertale - Time: 23:33:45
The Witcher 3: Wild Hunt - Time: 23:34:20
Divinity: Original Sin II - Definitive Edition - Time: 23:34:34
Final Fantasy XIV: Shadowbringers - Time: 23:34:38
Shadow of the Colossus - Time: 23:34:41
The Witcher 3: Wild Hunt - Blood and Wine - Time: 23:34:50
Celeste - Time: 23:35:04
INSIDE - Time: 23:35:07
NieR: Automata - Game of the YoRHa Edition - Time: 23:35:11
Resident Evil 2 - Time: 23:35:14
Dragon Quest XI S: Echoes of an Elusive Age - Definitive Edition - Time: 23:35:32
flower - Time: 23:35:34
Diablo III: Ultimate Evil Edition - Time: 23:35:38
Overwatch - Time: 23:

,console,video_game_name,summary,developer,genre(s),num_players,esrb_rating,critic_score,avg_user_score,user_review,user_score
0,ps4,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.6,"\nThis site is a joke, this the first time whe...",\n9\n
1,ps4,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.6,Fair review of RDR2\r I'm almost 15% finished ...,\n7\n
2,ps4,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.6,I really wanted to love it. The over-world is ...,\n6\n
3,ps4,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.6,"\nBeautiful graphics, excellent voice acting, ...",\n7\n
4,ps4,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.6,This game is really overrated.\rThe amazing en...,\n7\n
...,...,...,...,...,...,...,...,...,...,...,...
30753,ps4,F1 2017,"Win the 2017 World Championship, break every r...",Codemasters,"Genre(s): Racing, Simulation, Automobile",# of players: Up to 20,Rating: E,86,7.5,F1 2017 is not a perfect game. Specially the O...,\n9\n
30754,ps4,F1 2017,"Win the 2017 World Championship, break every r...",Codemasters,"Genre(s): Racing, Simulation, Automobile",# of players: Up to 20,Rating: E,86,7.5,"F1 2017 plays almost identical to F1 2016, plu...",\n6\n
30755,ps4,F1 2017,"Win the 2017 World Championship, break every r...",Codemasters,"Genre(s): Racing, Simulation, Automobile",# of players: Up to 20,Rating: E,86,7.5,I played this with a wheel and.... wow.\rThe p...,\n10\n
30756,ps4,F1 2017,"Win the 2017 World Championship, break every r...",Codemasters,"Genre(s): Racing, Simulation, Automobile",# of players: Up to 20,Rating: E,86,7.5,The 2017 chapter in the excellent Codemasters ...,\n10\n


In [10]:
scrape_top_100_games('xboxone')

Red Dead Redemption 2 - Time: 23:47:20
Grand Theft Auto V - Time: 23:47:46
Metal Gear Solid V: The Phantom Pain - Time: 23:47:53
Celeste - Time: 23:47:57
Resident Evil 2 - Time: 23:48:00
INSIDE - Time: 23:48:06
Forza Horizon 4 - Time: 23:48:10
Divinity: Original Sin II - Definitive Edition - Time: 23:48:18
What Remains of Edith Finch - Time: 23:48:20
Dragon Quest XI S: Echoes of an Elusive Age - Definitive Edition - Time: 23:48:22
The Witcher 3: Wild Hunt - Time: 23:48:25
Overwatch - Time: 23:48:44
Sekiro: Shadows Die Twice - Time: 23:48:48
Forza Horizon 3 - Time: 23:48:56
Dead Cells - Time: 23:49:03
Psychonauts 2 - Time: 23:49:05
F1 2020 - Time: 23:49:10
Monster Hunter: World - Iceborne - Time: 23:49:13
NieR: Automata - Become as Gods Edition - Time: 23:49:15
Monster Hunter: World - Time: 23:49:19
Ori and the Will of the Wisps - Time: 23:49:52
NBA 2K17 - Time: 23:50:03
Yakuza 0 - Time: 23:50:05
Mass Effect Legendary Edition - Time: 23:50:08
Yakuza: Like a Dragon - Time: 23:50:11
F1 20

,console,video_game_name,summary,developer,genre(s),num_players,esrb_rating,critic_score,avg_user_score,user_review,user_score
0,xboxone,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.1,This game is by means not a perfect game. The ...,\n9\n
1,xboxone,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.1,Red Dead Redemption 2 is an amazing game that ...,\n6\n
2,xboxone,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.1,Open world is not as open as BOTW or minecraft...,\n6\n
3,xboxone,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.1,"I just seems redundant now, leaving my review....",\n7\n
4,xboxone,Red Dead Redemption 2,Developed by the creators of Grand Theft Auto ...,Rockstar Games,"Genre(s): Action Adventure, Open-World",# of players: Up to 32,Rating: M,97,8.1,Here we have 2 games in 1: a wonderful open wo...,\n7\n
...,...,...,...,...,...,...,...,...,...,...,...
11734,xboxone,Planet Coaster: Console Edition,\nWelcome to a new era in coaster park simulat...,Frontier Developments,"Genre(s): Strategy, Management, Business / Tycoon",# of players: No Online Multiplayer,Rating: E,85,7.4,Planet Coaster is one of the better games of i...,\n8\n
11735,xboxone,Planet Coaster: Console Edition,\nWelcome to a new era in coaster park simulat...,Frontier Developments,"Genre(s): Strategy, Management, Business / Tycoon",# of players: No Online Multiplayer,Rating: E,85,7.4,\nTried this on gamepass and was really surpri...,\n9\n
11736,xboxone,Planet Coaster: Console Edition,\nWelcome to a new era in coaster park simulat...,Frontier Developments,"Genre(s): Strategy, Management, Business / Tycoon",# of players: No Online Multiplayer,Rating: E,85,7.4,I thought that this game was pretty good. I me...,\n9\n
11737,xboxone,Planet Coaster: Console Edition,\nWelcome to a new era in coaster park simulat...,Frontier Developments,"Genre(s): Strategy, Management, Business / Tycoon",# of players: No Online Multiplayer,Rating: E,85,7.4,"\nMe lembrou muito Screamride, o qual é muito ...",\n8\n


In [14]:
scrape_top_100_games('switch')

The Legend of Zelda: Breath of the Wild - Time: 08:18:18
Super Mario Odyssey - Time: 08:18:41
The House in Fata Morgana - Dreams of the Revenants Edition - - Time: 08:18:55
Tetris Effect: Connected - Time: 08:18:59
Hades - Time: 08:19:02
Divinity: Original Sin II - Definitive Edition - Time: 08:19:07
Ori and the Will of the Wisps - Time: 08:19:10
Undertale - Time: 08:19:12
Super Smash Bros. Ultimate - Time: 08:19:16
Celeste - Time: 08:19:42
Bayonetta 2 - Time: 08:19:47
Mario Kart 8 Deluxe - Time: 08:19:50
INSIDE - Time: 08:20:05
Dragon Quest XI S: Echoes of an Elusive Age - Definitive Edition - Time: 08:20:10
Sonic Mania Plus - Time: 08:20:15
SteamWorld Heist: Ultimate Edition - Time: 08:20:18
Shovel Knight: Treasure Trove - Time: 08:20:21
Animal Crossing: New Horizons - Time: 08:20:25
Ori and the Blind Forest: Definitive Edition - Time: 08:20:39
Bastion - Time: 08:20:42
Hollow Knight - Time: 08:20:44
Chicory: A Colorful Tale - Time: 08:20:52
Bayonetta + Bayonetta 2 - Time: 08:20:54
Xe

,console,video_game_name,summary,developer,genre(s),num_players,esrb_rating,critic_score,avg_user_score,user_review,user_score
0,switch,The Legend of Zelda: Breath of the Wild,Forget everything you know about The Legend of...,Nintendo,"Genre(s): Action Adventure, Open-World",# of players: No Online Multiplayer,Rating: E10+,97,8.7,+ Art Style - Beautiful and magical in a stunn...,\n10\n
1,switch,The Legend of Zelda: Breath of the Wild,Forget everything you know about The Legend of...,Nintendo,"Genre(s): Action Adventure, Open-World",# of players: No Online Multiplayer,Rating: E10+,97,8.7,\nSimply Amazing. Nuff said. This is the best ...,\n10\n
2,switch,The Legend of Zelda: Breath of the Wild,Forget everything you know about The Legend of...,Nintendo,"Genre(s): Action Adventure, Open-World",# of players: No Online Multiplayer,Rating: E10+,97,8.7,\nThe fact that there are so many people hatin...,\n10\n
3,switch,The Legend of Zelda: Breath of the Wild,Forget everything you know about The Legend of...,Nintendo,"Genre(s): Action Adventure, Open-World",# of players: No Online Multiplayer,Rating: E10+,97,8.7,\nSimply breathtaking and stunning. Nintendo h...,\n10\n
4,switch,The Legend of Zelda: Breath of the Wild,Forget everything you know about The Legend of...,Nintendo,"Genre(s): Action Adventure, Open-World",# of players: No Online Multiplayer,Rating: E10+,97,8.7,\nEste juego es la perfeccion en cuanto a jueg...,\n10\n
...,...,...,...,...,...,...,...,...,...,...,...
14549,switch,The Binding of Isaac: Afterbirth +,The latest expansion to The Binding of Isaac: ...,Edmund McMillen,"Genre(s): Action, Shooter, Shoot-'Em-Up, Top-Down",# of players: No Online Multiplayer,Rating: M,85,8.3,\nFinally a game that makes my switch worth pl...,\n9\n
14550,switch,The Binding of Isaac: Afterbirth +,The latest expansion to The Binding of Isaac: ...,Edmund McMillen,"Genre(s): Action, Shooter, Shoot-'Em-Up, Top-Down",# of players: No Online Multiplayer,Rating: M,85,8.3,\nyou will be always have curiosity to play th...,\n10\n
14551,switch,The Binding of Isaac: Afterbirth +,The latest expansion to The Binding of Isaac: ...,Edmund McMillen,"Genre(s): Action, Shooter, Shoot-'Em-Up, Top-Down",# of players: No Online Multiplayer,Rating: M,85,8.3,\nOH EGG FATHER PLEASE LEAD ME TO MORE AMAZING...,\n10\n
14552,switch,The Binding of Isaac: Afterbirth +,The latest expansion to The Binding of Isaac: ...,Edmund McMillen,"Genre(s): Action, Shooter, Shoot-'Em-Up, Top-Down",# of players: No Online Multiplayer,Rating: M,85,8.3,\nMy favorite game of all time. I have spent o...,\n10\n


In [15]:
scrape_top_100_games('pc')

Disco Elysium: The Final Cut - Time: 08:28:16
Half-Life 2 - Time: 08:28:20
Grand Theft Auto V - Time: 08:28:38
The Orange Box - Time: 08:28:54
Half-Life - Time: 08:29:00
BioShock - Time: 08:29:17
Baldur's Gate II: Shadows of Amn - Time: 08:29:42
Portal 2 - Time: 08:29:49
The Elder Scrolls V: Skyrim - Time: 08:30:22
Mass Effect 2 - Time: 08:30:51
Elden Ring - Time: 08:31:13
Grand Theft Auto: Vice City - Time: 08:31:38
Sid Meier's Civilization II - Time: 08:31:46
Quake - Time: 08:31:49
BioShock Infinite - Time: 08:31:54
The Elder Scrolls IV: Oblivion - Time: 08:32:21
Grim Fandango - Time: 08:32:37
Diablo - Time: 08:32:39
Sid Meier's Civilization IV - Time: 08:32:43
The Witcher 3: Wild Hunt - Time: 08:32:50
Company of Heroes - Time: 08:33:27
Half-Life: Alyx - Time: 08:33:36
Divinity: Original Sin II - Time: 08:34:00
Unreal Tournament 2004 - Time: 08:34:16
Starcraft II: Wings of Liberty - Time: 08:34:21
Minecraft - Time: 08:34:49
Red Dead Redemption 2 - Time: 08:35:16
Grand Theft Auto III 

,console,video_game_name,summary,developer,genre(s),num_players,esrb_rating,critic_score,avg_user_score,user_review,user_score
0,pc,Disco Elysium: The Final Cut,Disco Elysium - The Final Cut is the definitiv...,ZA/UM,"Genre(s): Role-Playing, General, Western-Style",# of players: No Online Multiplayer,Rating: M,97,8.2,"\nFinally, those looking for something that tr...",\n10\n
1,pc,Disco Elysium: The Final Cut,Disco Elysium - The Final Cut is the definitiv...,ZA/UM,"Genre(s): Role-Playing, General, Western-Style",# of players: No Online Multiplayer,Rating: M,97,8.2,\nCommunist propaganda garbage. Not worth play...,\n0\n
2,pc,Disco Elysium: The Final Cut,Disco Elysium - The Final Cut is the definitiv...,ZA/UM,"Genre(s): Role-Playing, General, Western-Style",# of players: No Online Multiplayer,Rating: M,97,8.2,The game's strengths come from the intrigue of...,\n5\n
3,pc,Disco Elysium: The Final Cut,Disco Elysium - The Final Cut is the definitiv...,ZA/UM,"Genre(s): Role-Playing, General, Western-Style",# of players: No Online Multiplayer,Rating: M,97,8.2,"The writing, the story, the world building and...",\n8\n
4,pc,Disco Elysium: The Final Cut,Disco Elysium - The Final Cut is the definitiv...,ZA/UM,"Genre(s): Role-Playing, General, Western-Style",# of players: No Online Multiplayer,Rating: M,97,8.2,I spent 41 hours playing this game from start ...,\n7\n
...,...,...,...,...,...,...,...,...,...,...,...
32712,pc,Spelunky,\nSpelunky is a unique platformer with randomi...,Derek Yu,"Genre(s): Action, Platformer, Platformer, 2D, 2D",# of players: No Online Multiplayer,NaN,90,7.4,This game has a great concept and its mechanic...,\n6\n
32713,pc,Spelunky,\nSpelunky is a unique platformer with randomi...,Derek Yu,"Genre(s): Action, Platformer, Platformer, 2D, 2D",# of players: No Online Multiplayer,NaN,90,7.4,\nShort entertaining roguelike puzzle-ish plat...,\n8\n
32714,pc,Spelunky,\nSpelunky is a unique platformer with randomi...,Derek Yu,"Genre(s): Action, Platformer, Platformer, 2D, 2D",# of players: No Online Multiplayer,NaN,90,7.4,\nOne of my all-time favourite games. So much ...,\n10\n
32715,pc,Spelunky,\nSpelunky is a unique platformer with randomi...,Derek Yu,"Genre(s): Action, Platformer, Platformer, 2D, 2D",# of players: No Online Multiplayer,NaN,90,7.4,"The most perfect ""video game-y"" video game. Ev...",\n10\n


In [17]:
scrape_top_100_games('xbox-series-x')

In [18]:
scrape_top_100_games('ps5')